In [1]:
import re, json, time
from selenium import webdriver
import os

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


# Tratamento


In [2]:
with open("canais.txt", "r") as f:
    # read each line as a item in a list and remove the \n
    canais = [line.strip() for line in f.readlines()]


def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector

In [3]:
import re


import locale

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

base_path = f"./data/{canais[0].split('@')[-1]}/"

print(base_path)

./data/EspacolaserNews/


In [4]:
def trata_inscritos(col: pd.Series) -> pd.Series:
    col = col.split(" inscritos")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    if "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )

    return int(col.replace(" ", "").replace(",", "."))


def trata_visualizacoes(col):
    # treat the characters U+00a0

    # if its a numeric value return it
    if type(col) == int or type(col) == float:
        return col

    col = col.replace("\xa0", "")

    col = col.split(" visualizações")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )

    try:
        return int(col.replace(" ", "").replace(".", "").replace(",", "."))
    except:
        return 0


def trata_comments(col):
    # print(col)
    # print(type(col))

    if type(col) == int or type(col) == float:
        return col

    value = 0

    try:
        value = int(col)
    except:
        pass

    if "Transcrição" in col:
        return 0

    col = col.replace("\xa0", "")

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        try:
            value = int(
                float(col.replace("mi", "").replace(" ", "").replace(",", "."))
                * 1000000
            )
        except:
            value = 0

    return (
        int(col.replace(" ", "").replace(".", "").replace(",", "."))
        if value != 0
        else 0
    )


def trata_data(value):
    meses = {
        "jan": "01",
        "fev": "02",
        "mar": "03",
        "abr": "04",
        "mai": "05",
        "jun": "06",
        "jul": "07",
        "ago": "08",
        "set": "09",
        "out": "10",
        "nov": "11",
        "dez": "12",
    }
    series_data = re.sub(r"\. de", "", value)
    # print(series_data)
    series_data = re.search(r"(\d{1,2} de [a-zA-Z]+ \d{4})", series_data).group(0)
    # replace month
    for k, v in meses.items():
        series_data = series_data.replace(k, v)

    # print(series_data)
    series_data = pd.to_datetime(series_data, format="%d de %m %Y", errors="coerce")
    return series_data

trata videos

In [5]:
videos = pd.read_csv(base_path + "videos.csv")

display(videos)
display(videos.columns)

,title,views,likes,date,comments_amount,channel_name,subscribers
0,NaN,NaN,0,NaN,NaN,Espaçolaser,"14,3 mil inscritos"
1,Espaçolaser: Depilação ainda é um pesadelo par...,"1,8 mil visualizações",8,23 de out. de 2023,NaN,Espaçolaser,"14,3 mil inscritos"
2,Outubro OFF Espaçolaser,"1,1 mil visualizações",0,29 de set. de 2023,NaN,Espaçolaser,"14,3 mil inscritos"
3,Minha História de Amor Espaçolaser,362 visualizações,7,15 de ago. de 2023,2,Espaçolaser,"14,3 mil inscritos"
4,Minha História de Amor Espaçolaser,182 visualizações,7,15 de ago. de 2023,Transcrição,Espaçolaser,"14,3 mil inscritos"
5,Transformando Vidas Rosangela,18 visualizações,0,26 de out. de 2023,Transcrição,GiOlaser | Inspirada em você,732 inscritos
6,Polineiry e Alisson Transformando Vidas,49 visualizações,0,26 de out. de 2023,Transcrição,GiOlaser | Inspirada em você,732 inscritos
7,Cuidado de pai para filho,122 visualizações,5,26 de out. de 2023,Transcrição,GiOlaser | Inspirada em você,732 inscritos
8,Transformando Vidas Faustino e Vanessa,15 visualizações,0,26 de out. de 2023,Transcrição,GiOlaser | Inspirada em você,732 inscritos
9,GiOlaser Flagship - Anália Franco | Grande Ina...,146 visualizações,5,25 de ago. de 2023,NaN,GiOlaser | Inspirada em você,732 inscritos


Index(['title', 'views', 'likes', 'date', 'comments_amount', 'channel_name',
       'subscribers'],
      dtype='object')

In [6]:
videos.dropna(subset="title").assign(
    # subscribers=lambda df_: df_["subscribers"].apply(trata_inscritos),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    date=lambda df_: df_["date"].apply(trata_data),
    # comments_amount = lambda df_: df_["comments_amount"].apply(trata_comments),
    category="RECENT",
).to_parquet(f"{base_path}videos.parquet")

# Trata Top videos

In [7]:
import pandas as pd


top_videos = pd.read_csv(base_path + "top_videos.csv")

display(top_videos.tail())

,title,views,likes,date,comments_amount,channel_name
10,ONOCAST com Edna Onodera e Dani Magagna. | Cri...,264 visualizações,0,Estreou em 17 de abr. de 2023,1,Onodera Estética
11,NaN,NaN,0,NaN,NaN,Onodera Estética
12,ONOCAST com Edna Onodera e Dani Magagna. | Cac...,115 visualizações,0,Estreou em 17 de mai. de 2023,Transcrição,Onodera Estética
13,ONODERA 42 ANOS: CONHEÇA A TRAJETÓRIAS DESSAS ...,102 visualizações,4,5 de abr. de 2023,Transcrição,Onodera Estética
14,ONOCAST com Edna Onodera e Dani Magagna. | Fê ...,97 visualizações,0,18 de jul. de 2023,1,Onodera Estética


In [8]:
top_videos.dropna().assign(
    date=lambda df_: df_["date"].apply(trata_data),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    # comments_amount = lambda df_: df_["comments_amount"].apply(trata_comments),

    category="TOP",
).to_parquet(base_path + "top_videos.parquet")

In [9]:
top_videos_silver = pd.read_parquet(base_path + "top_videos.parquet")

In [10]:
# # Trata lifetime


df_lifetime = pd.read_csv(f"{base_path}lifetime.csv")


df_lifetime.head()


def days_on_youtube(value) -> int:
    import datetime

    today = datetime.date.today()
    return (today - value.date()).days

In [11]:
df_lifetime

,channel_title,date_joined,total_views,total_subs,total_videos
0,Espaçolaser,Inscreveu-se em 23 de abr. de 2018,2.296.857 visualizações,"14,3 mil inscritos",109
1,GiOlaser | Inspirada em você,Inscreveu-se em 12 de dez. de 2017,484.091 visualizações,732 inscritos,80
2,Onodera Estética,Inscreveu-se em 20 de mar. de 2023,1.277 visualizações,45 inscritos,19


In [12]:
df_lifetime = df_lifetime.assign(
    date_joined=lambda df_: df_["date_joined"].apply(trata_data),
    total_views=lambda df_: df_["total_views"].apply(trata_visualizacoes),
    total_videos=lambda d: d["total_videos"].apply(trata_visualizacoes),
    total_subs=lambda df_: df_["total_subs"].apply(trata_inscritos),
    days_since_joined=lambda df_: df_["date_joined"].apply(days_on_youtube),
)  # .to_csv(base_path + "lifetime_clean.csv", index=False)

In [13]:
lifetime_csv = pd.DataFrame()

lifetime_csv["Channel Id"] = np.arange(5)

lifetime_csv[" Channel Name"] = df_lifetime["channel_title"]
lifetime_csv[" Days on YouTube"] = df_lifetime["days_since_joined"]
lifetime_csv[" Total Views"] = df_lifetime["total_views"]
lifetime_csv[" Views per Day"] = (
    lifetime_csv[" Total Views"] / lifetime_csv[" Days on YouTube"]
)
lifetime_csv[" Total Subscribers"] = df_lifetime["total_subs"]
lifetime_csv[" Subscribers per Day"] = (
    lifetime_csv[" Total Subscribers"] / lifetime_csv[" Days on YouTube"]
)
lifetime_csv[" Total Uploads"] = df_lifetime["total_videos"]
lifetime_csv[" Uploads per Month"] = lifetime_csv[" Total Uploads"] / (
    lifetime_csv[" Days on YouTube"] / 30
)
lifetime_csv[""] = np.arange(5)

lifetime_csv.to_csv(base_path + "Lifetime_.csv", index=False, decimal=",")

In [14]:
videos = pd.read_parquet(f"{base_path}videos.parquet")
top_videos = pd.read_parquet(f"{base_path}top_videos.parquet")

In [15]:
videos = videos.assign(title_canal=lambda d: d["channel_name"] + " - " + d["title"])

top_videos = top_videos.assign(
    title_canal=lambda d: d["channel_name"] + " - " + d["title"]
)

In [16]:
videos_aux = pd.DataFrame()


videos_aux["Channel Id"] = np.arange(1, len(videos) + 1)
videos_aux[" Channel Name"] = videos["channel_name"]
videos_aux[" Video Id"] = np.arange(1, len(videos) + 1)
videos_aux[" VideoUrl"] = np.arange(1, len(videos) + 1)
videos_aux[" Title"] = videos["title_canal"]
videos_aux[" Published Date"] = videos["date"]
videos_aux[" Views"] = videos["views"]
videos_aux[" Likes"] = videos["likes"]
videos_aux[" Dislikes"] = np.arange(1, len(videos) + 1)
videos_aux[" Comments"] = videos["comments_amount"]
videos_aux[" Engagement %"] = np.arange(1, len(videos) + 1)

In [17]:
videos_aux.to_csv(base_path + "RecentUploads.csv", index=False)

In [18]:
videos_aux = pd.DataFrame()


videos_aux["Channel Id"] = np.arange(1, len(videos) + 1)
videos_aux[" Channel Name"] = top_videos["channel_name"]
videos_aux[" Video Id"] = np.arange(1, len(videos) + 1)
videos_aux[" VideoUrl"] = np.arange(1, len(videos) + 1)
videos_aux[" Title"] = top_videos["title_canal"]
videos_aux[" Published Date"] = top_videos["date"]
videos_aux[" Views"] = top_videos["views"]
videos_aux[" Likes"] = top_videos["likes"]
videos_aux[" Dislikes"] = np.arange(1, len(videos) + 1)
videos_aux[" Comments"] = top_videos["comments_amount"]
videos_aux[" Engagement %"] = np.arange(1, len(videos) + 1)

In [19]:
videos_aux.to_csv(base_path + "TopUploads.csv", index=False)

last 30 days

In [20]:
# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

# print(canais)

base_url = "https://socialblade.com/youtube/channel/"

last_30days = pd.read_parquet(f'data/{canais[0].split("@")[-1]}/last_30_days.parquet')


def fix_views(view: pd.Series) -> pd.Series:
    view = view.str.replace("+", "").str.replace(",", "")
    # print(view)

    view = view.apply(lambda x: float(x.replace("K", "")) * 1000 if "K" in x else x)
    view = view.apply(
        lambda x: float(x.replace("M", "")) * 1000000 if "M" in str(x) else x
    )

    return view


last_30days = last_30days.assign(
    views=lambda x: fix_views(x["views"]), subs=lambda x: fix_views(x["subs"])
)

# Replace '---' with NaN in the 'subs' column
last_30days["subs"] = last_30days["subs"].replace("--", np.nan)

In [21]:
last_30days = last_30days.astype({"views": "float64", "subs": "float64"})

In [22]:
last_30_days_aux = pd.DataFrame()


last_30_days_aux["Channel Id"] = np.arange(5)
last_30_days_aux[" Channel Name"] = df_lifetime["channel_title"]
last_30_days_aux["  Views Last 30 Days"] = last_30days["views"]
last_30_days_aux[" Views per Day"] = last_30_days_aux["  Views Last 30 Days"] / 30
last_30_days_aux[" Total Subscribers Last 30 Days"] = last_30days["subs"]
last_30_days_aux[" Subscribers per Day"] = (
    last_30_days_aux[" Total Subscribers Last 30 Days"] / 30
)
last_30_days_aux[" Total Uploads Last 30 Days"] = [
    "ATUALIZAR",
    "ATUALIZAR",
    "ATUALIZAR",
    "ATUALIZAR",
    "ATUALIZAR",
]
last_30_days_aux[" Uploads per Week"] = np.arange(5)
last_30_days_aux[""] = np.arange(5)

In [23]:
last_30_days_aux

,Channel Id,Channel Name,Views Last 30 Days,Views per Day,Total Subscribers Last 30 Days,Subscribers per Day,Total Uploads Last 30 Days,Uploads per Week,
0,0,Espaçolaser,15125.0,504.166667,200.0,6.666667,ATUALIZAR,0,0
1,1,GiOlaser | Inspirada em você,1406.0,46.866667,12.0,0.400000,ATUALIZAR,1,1
2,2,Onodera Estética,47.0,1.566667,1.0,0.033333,ATUALIZAR,2,2
3,3,NaN,NaN,NaN,NaN,NaN,ATUALIZAR,3,3
4,4,NaN,NaN,NaN,NaN,NaN,ATUALIZAR,4,4


In [24]:
last_30_days_aux.to_csv(base_path + "Last30Days.csv", index=False, decimal=",")

In [25]:
# last_30days.to_csv(f'data/{canais[0].split("@")[-1]}/last_30days.csv', index=False)

In [26]:
def A(x):
    return len(x)


def B(x):
    vetor = []
    for i in range(x):
        vetor.append(i)

    return vetor


lista = [-1, 1, -2, 3, -5, 8]
print(B(A(lista)))

[0, 1, 2, 3, 4, 5]
